In [1]:

# Aktuell bestes Modell aufbauen 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from keras import layers
import matplotlib.pyplot as plt
import kerastuner as kt
import shutil
import os
import json
import gc
from sklearn.metrics import mean_absolute_error


def bestes_model(X_train_scaled, X_val_scaled, X_test_scaled, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels):
      
    Y_train = np.squeeze(Y_train)
    Y_test = np.squeeze(Y_test)
    Y_val =np.squeeze(Y_val)
    Y_train_scaled = np.squeeze(Y_train_scaled)
    Y_val_scaled = np.squeeze(Y_val_scaled)
    
    # TensorFlow-Sitzung zurücksetzen
    keras.backend.clear_session()
    gc.collect()

# CNN Modell definieren
    input_layer = layers.Input(shape=(10,11))

    #Hyperparametersuche 2 60 Trials
    conv_1 = layers.Conv1D(filters=160, kernel_size=2, activation='relu', padding='same', kernel_regularizer=keras.regularizers.l2(0.01))(input_layer)
    pool_1 = layers.MaxPooling1D(pool_size=1)(conv_1)
    conv_2 = layers.Conv1D(filters=480, kernel_size=4, activation='relu', padding='same', strides=1, kernel_regularizer=keras.regularizers.l2(0.01))(pool_1)
    pool_2 = layers.MaxPooling1D(pool_size=2)(conv_2)

    flatten = layers.Flatten()(pool_2)

    # print(flatten)
    # print(type(flatten))
    # print(flatten.shape)
    
    #Hyperparametersuche 3
    dense_layer = layers.Dense(64, activation='tanh', kernel_regularizer=keras.regularizers.l2(0.01))(flatten)

    # Output Layers definieren
    X_output = layers.Dense(1, activation='linear', name='Verstellweg_X')(dense_layer)
    Y_output = layers.Dense(1, activation='linear', name='Verstellweg_Y')(dense_layer)
    Phi_output = layers.Dense(1, activation='linear', name='Verstellweg_Phi')(dense_layer)

    # Liste erstellen für alle Outputs
    outputs = [X_output, Y_output, Phi_output]

    # Modell definieren 
    model = keras.Model(inputs=input_layer, outputs=outputs)
    
    # # Hyperparameter 3 60 Trials
    model.compile(optimizer=keras.optimizers.Adam(0.0003255639325303961), 
                loss=['mean_absolute_error', 'mean_absolute_error', 'mean_absolute_error'], 
                metrics={'Verstellweg_X': 'mae', 'Verstellweg_Y': 'mae', 'Verstellweg_Phi': 'mae'})
    
    # Modell zusammenfassen
    #model.summary()

    # Define early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

    CNN = model.fit(X_train_scaled, [Y_train[:,0], Y_train[:,1], Y_train[:,2]],
                            epochs=20,
                            validation_data=(X_val_scaled, [Y_val[:, 0], Y_val[:, 1], Y_val[:, 2]]),
                            callbacks=[early_stopping])
    
    # Vorhersagen für den Testdatensatz erstellen
    # predictions = model.predict(X_test_scaled)
    # X_p, Y_p, Phi_p = predictions
    
    #Falls mit KNR oder RFR probiert werden muss 
    def get_flatten_layer(model):
        # Iterate through the layers of the model to find the first Flatten layer
        for layer in model.layers:
            if isinstance(layer, keras.layers.Flatten):
                return layer
        raise ValueError("No Flatten layer found in the model.")


# Get the Flatten layer
    flatten_layer = get_flatten_layer(model)
    # Create an intermediate model to get the flattened layer
    intermediate_model = keras.Model(inputs=model.input, outputs=flatten_layer.output)

    # # Extract features
    X_train_flat = intermediate_model.predict(X_train_scaled)
    # X_val_flat = intermediate_model.predict(X_val_scaled)
    X_test_flat = intermediate_model.predict(X_test_scaled)
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.neighbors import KNeighborsRegressor

    print(X_train_flat.shape)
    #print(X_val_flat.shape)
    print(X_test_flat.shape)
    print(X_train_flat)

    #Model = RandomForestRegressor(n_estimators=800, max_depth=20, min_samples_split=2, min_samples_leaf=1, max_features='sqrt', bootstrap=False)
    Model = KNeighborsRegressor(algorithm= 'auto', n_neighbors= 3, weights= 'distance', metric='manhattan')

    Model.fit(X_train_flat, Y_train)

    #Y_pred_val = Model.predict(X_val_flat)
    Y_test_pred = Model.predict(X_test_flat)
    print(Y_test_pred.shape)
    
    X_p = Y_test_pred[:, 0]
    Y_p = Y_test_pred[:, 1]
    Phi_p = Y_test_pred[:, 2]
    
    print(X_p.shape)
    
    
     # Error für jedes Label berechnen
    Fehler_X = Y_test[:,0]-X_p
    Fehler_Y = Y_test[:,1]-Y_p
    Fehler_Phi = Y_test[:,2]-Phi_p
    
 
    # Fehler_X = Y_test[:,0]-X_p[:,0]
    # Fehler_Y = Y_test[:,1]-Y_p[:,0]
    # Fehler_Phi = Y_test[:,2]-Phi_p[:,0]
    
    print(Fehler_X.shape)
    print(Fehler_X)
    
    
    # Fehler in einen DataFrame konvertieren
    df_Fehler = pd.DataFrame({
        'Label_X': Y_test[:,0],
        'Label_Y': Y_test[:,1],
        'Label_Phi': Y_test[:,2],
        'Fehler_X': Fehler_X,
        'Fehler_Y': Fehler_Y,
        'Fehler_Phi': Fehler_Phi})
    

    #print(X_train_flat.shape)
    #print(X_val_flat.shape)
        #print(predictions)
    mae_X = mean_absolute_error(Y_test[:, 0], X_p)
    mae_Y = mean_absolute_error(Y_test[:, 1], Y_p)
    mae_Phi = mean_absolute_error(Y_test[:, 2], Phi_p)

    # X und Y kombinieren da diese zusammen sakliert werden
    # XY_p = np.column_stack((X_p, Y_p))

    # Rückskalierung der Vorhersagen
    # XY_pred = scaler_Y_mm.inverse_transform(XY_p)
    # X_pred, Y_pred = XY_pred[:, 0], XY_pred[:, 1]
    # Phi_pred = scaler_Y_phi.inverse_transform(Phi_p.reshape(-1, 1)).flatten()
    
    # MAE für den Testdatensatz berechnen
    # mae_X = mean_absolute_error(Y_test[:,0], X_p)
    # mae_Y = mean_absolute_error(Y_test[:,1], Y_p)
    # mae_Phi = mean_absolute_error(Y_test[:,2], Phi_p)

    print(f"Mean Absolute Error for Verstellweg_X: {mae_X}")
    print(f"Mean Absolute Error for Verstellweg_Y: {mae_Y}")
    print(f"Mean Absolute Error for Verstellweg_Phi: {mae_Phi}")
    
    return mae_X, mae_Y, mae_Phi, df_Fehler

C:\Users\corvi\AppData\Local\Temp\ipykernel_30568\2292960643.py:10: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
import random 
from Load_Data_for_Modelling_Function import Data_for_Model
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from Fensterung_Scaling_CNN_ValData_Interpolation_Test import Fensterung_Scale
from sklearn.metrics import mean_absolute_error
import pandas as pd


random.seed(2)
# Generieren einer Liste von 10 eindeutigen zufälligen Ganzzahlen zwischen 0 und 100
Random_numbers = random.sample(range(101), 10)
print(Random_numbers)


data = Data_for_Model(0,1800)

Liste_Fehler_Blechsplit = []
Liste_Fehler_Standardsplit= []

Liste_MAEs_Blechsplit =[]
Liste_MAEs_Standardsplit = []

MAE_StandardSplit_leer = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])
MAE_BlechSplit_leer = pd.DataFrame(columns=['CV', 'Datentyp','Error', 'X', 'y', 'phi'])

for n in Random_numbers: 
    
    X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test = Fensterung_Scale(data, Validation_data=1, random=n, Train_Test_Split =1, size=0.2)
    mae_X, mae_Y, mae_phi, df_Fehler = bestes_model(X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels)
    
    MAE_StandardSplit_df = pd.DataFrame([{'CV':n, 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': mae_X, 'y': mae_Y, 'phi': mae_phi}])
    Liste_MAEs_Standardsplit.append(MAE_StandardSplit_df)
    
    df_Fehler.insert(loc=0, column='SplitMethode', value='Standardsplit')
    df_Fehler.insert(loc=1, column='CV', value=n)
    Liste_Fehler_Standardsplit.append(df_Fehler)
    
MAE_Standardsplit = pd.concat(Liste_MAEs_Standardsplit, ignore_index=True)
Fehler_Standardsplit_df = pd.concat(Liste_Fehler_Standardsplit, ignore_index=True)

# for n in Random_numbers:
    
#     X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels, Angepasste_Blechnummern_test   = Fensterung_Scale(data, Validation_data=1, random=n, Train_Test_Split =2, size=0.2)
#     mae_X, mae_Y, mae_phi, df_Fehler = bestes_model(X_train, X_val, X_test, Y_train_scaled, Y_val_scaled, Y_test_scaled, Y_train, Y_val, Y_test, scalers_features, scaler_labels)
    
#     MAE_BlechSplit_df = pd.DataFrame([{'CV':n, 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': mae_X, 'y': mae_Y, 'phi': mae_phi}])
#     Liste_MAEs_Blechsplit.append(MAE_BlechSplit_df)
    
#     df_Fehler.insert(loc=0, column='SplitMethode', value='Blechsplit')
#     df_Fehler.insert(loc=1, column='CV', value=n)
#     Liste_Fehler_Blechsplit.append(df_Fehler)
    
# MAE_Blechsplit = pd.concat(Liste_MAEs_Blechsplit, ignore_index=True)
# Fehler_Blechsplit_df = pd.concat(Liste_Fehler_Blechsplit, ignore_index=True)
    
    
# Ausgabe der DataFrames
# print("MAE Standard Split:")
# print(MAE_StandardSplit_df)
print("\nMAE Blech Split:")
print(MAE_Standardsplit)

[7, 11, 10, 46, 21, 94, 85, 39, 32, 77]
Anzahl der Zeilen im finalen DataFrame: 255600
None
142
254180
(25418, 1, 3)
Shape für die Features der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (203344, 10, 11)
Shape für die Features der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25418, 10, 11) 
Shape für die Labels der gesamten Trainingsdaten, also im Falle einer Interpolation mit allen Daten (203344, 1, 3)
Shape für die Labels der gesamten Testdaten, also im Falle einer Interpolation mit allen Daten (25418, 1, 3)

Epoch 1/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 75s 11ms/step - Verstellweg_Phi_mae: 0.5700 - Verstellweg_X_mae: 0.0424 - Verstellweg_Y_mae: 0.4596 - loss: 2.2087 - val_Verstellweg_Phi_mae: 0.2893 - val_Verstellweg_X_mae: 0.0097 - val_Verstellweg_Y_mae: 0.2301 - val_loss: 0.8128
Epoch 2/20
6355/6355 ━━━━━━━━━━━━━━━━━━━━ 83s 13ms/step - Verstellweg_Phi_mae: 0.2639 - Verstellweg_X_mae: 0.0082 - Verstellweg_Y_mae: 0.1977 - loss: 0

In [3]:
Ordner = r'C:\Users\corvi\OneDrive - stud.tu-darmstadt.de\Desktop\Masterthesis\13_ExcelvonDaten_Code\DeepLearning\CNN_KNR\RandomSeed2'

# print(Fehler_Blechsplit_df)
# print(MAE_Blechsplit)
Mean_Standard = MAE_Standardsplit[['X','y','phi']].mean() 
Mean_Blech = MAE_BlechSplit_leer[['X','y','phi']].mean() 
Std_Standard = MAE_Standardsplit[['X','y','phi']].std()
Std_Blech = MAE_BlechSplit_leer[['X','y','phi']].std()

MAE_StandardSplit_df = pd.concat([MAE_Standardsplit, pd.DataFrame([{'CV': 'Mittelwert', 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': Mean_Standard[0], 'y': Mean_Standard[1], 'phi': Mean_Standard[2]}])], ignore_index=True)
MAE_BlechSplit = pd.concat([MAE_BlechSplit_leer, pd.DataFrame([{'CV': 'Mittelwert', 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': Mean_Blech[0], 'y': Mean_Blech[1], 'phi': Mean_Blech[2]}])], ignore_index=True)
MAE_StandardSplit_comp = pd.concat([MAE_StandardSplit_df, pd.DataFrame([{'CV': 'Standardabweichung', 'Datentyp': 'Standardsplit', 'Error' : 'MAE', 'X': Std_Standard[0], 'y': Std_Standard[1], 'phi': Std_Standard[2]}])], ignore_index=True)
MAE_BlechSplit_comp = pd.concat([MAE_BlechSplit, pd.DataFrame([{'CV': 'Standardabweichung', 'Datentyp': 'Blechsplit', 'Error' : 'MAE', 'X': Std_Blech[0], 'y': Std_Blech[1], 'phi': Std_Blech[2]}])], ignore_index=True)

# print(MAE_BlechSplit)
# print(MAE_StandardSplit)

Errors_for_CSV = pd.concat([MAE_StandardSplit_comp, MAE_BlechSplit_comp], axis=1)
print(Errors_for_CSV['X'])

Errors_for_CSV.columns = ['CV', 'Datentyp', 'Error', 'X', 'y', 'phi', 'CV1',
       'Datentyp1', 'Error1', 'X1', 'y1', 'phi1']

for Column in Errors_for_CSV.columns:
        Errors_for_CSV[Column] = Errors_for_CSV[Column].astype(str).str.replace('.', ',')

for Column in MAE_BlechSplit_comp.columns:
         MAE_BlechSplit_comp[Column] = MAE_BlechSplit_comp[Column].astype(str).str.replace('.', ',')
         
for Column in Fehler_Standardsplit_df:
        Fehler_Standardsplit_df[Column] = Fehler_Standardsplit_df[Column].astype(str).str.replace('.', ',')

Fehler_Standardsplit_df.to_csv(f'{Ordner}\Fehler_CNN_KNR_Blechsplit_Randomseed2.csv', index=True, sep=';')
        
# for Column in Fehler_Blechsplit_df:
#         Fehler_Blechsplit_df[Column] = Fehler_Blechsplit_df[Column].astype(str).str.replace('.', ',')

Fehler_Standardsplit_df.to_csv(f'{Ordner}\Fehler_CNN_KNR_Standardsplit_Randomseed2.csv', index=True, sep=';')

Errors_for_CSV.to_csv(f'{Ordner}\CNN_KNR_Standardsplit_Randomseed2.csv', index=True, sep=';')

               X    X
0   5.905349e-18  NaN
1   6.219835e-18  NaN
2   6.115006e-18  NaN
3   5.660748e-18  NaN
4   6.429492e-18  NaN
5   6.080063e-18  NaN
6   6.254778e-18  NaN
7   6.604207e-18  NaN
8   6.045120e-18  NaN
9   4.996834e-18  NaN
10  6.031143e-18  NaN
11  4.478510e-19  NaN


C:\Users\corvi\AppData\Local\Temp\ipykernel_30568\2870975324.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Errors_for_CSV[Column] = Errors_for_CSV[Column].astype(str).str.replace('.', ',')
C:\Users\corvi\AppData\Local\Temp\ipykernel_30568\2870975324.py:28: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  MAE_BlechSplit_comp[Column] = MAE_BlechSplit_comp[Column].astype(str).str.replace('.', ',')
C:\Users\corvi\AppData\Local\Temp\ipykernel_30568\2870975324.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Fehler_Sta